In [1]:
import itertools
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import json


# Definición de los niveles para cada factor:
# Por ejemplo, estos son valores de prueba; se pueden ajustar según las necesidades.
niveles_sesgo     = [0, 0.5, 1.0, 1.5, 2.0]        # Desplazamiento de la media
niveles_curtosis  = [2, 3, 4, 5, 6]                  # Curtosis deseada (placeholder)
niveles_varianza  = [0.5, 1, 1.5, 2, 2.5]            # Varianzas (se usa para calcular la desviación)
niveles_outliers  = [0, 1, 2, 3, 4]                  # Cantidad de outliers a inyectar
niveles_tamano    = [50, 100, 200, 500, 1000]         # Tamaño muestral

# Generamos todas las combinaciones de parámetros (3125 corridas)
combinaciones = list(itertools.product(niveles_sesgo, niveles_curtosis, niveles_varianza, niveles_outliers, niveles_tamano))
print(f"Total de corridas: {len(combinaciones)}")

def generar_datos(sesgo, curtosis, varianza, outliers, tamano):
    """
    Genera un conjunto de datos simulados a partir de una distribución normal, 
    ajustando:
      - 'sesgo': se añade a los datos para desplazar la media.
      - 'curtosis': (placeholder) se podría aplicar una transformación para ajustar la curtosis.
      - 'varianza': se usa para definir la dispersión.
      - 'outliers': se inyectan valores extremos multiplicando algunos datos.
      - 'tamano': cantidad de datos en la muestra.
    """
    # Generar datos con distribución normal de media 0 y desviación sqrt(varianza)
    data = np.random.normal(loc=0, scale=np.sqrt(varianza), size=tamano)
    
    # Aplicar sesgo desplazando la media
    data = data + sesgo
    
    # Ajuste de curtosis: por ahora se deja como placeholder
    # Ejemplo: data = transformar_curtosis(data, curtosis)
    
    # Introducir outliers de forma sencilla: se seleccionan índices aleatorios y se multiplican sus valores
    if outliers > 0 and outliers < tamano:
        indices = np.random.choice(range(tamano), size=outliers, replace=False)
        data[indices] = data[indices] * 10  # Factor para simular outliers
    
    return data

def test_normalidad(data):
    """
    Aplica 5 pruebas de normalidad:
      - Kolmogorov-Smirnov (KS)
      - Shapiro-Wilk
      - D'Agostino-Pearson
      - Anderson-Darling
      - Análisis cuantitativo del Q-Q (regresión lineal sobre los cuantiles)
    Retorna un diccionario con los resultados de cada prueba.
    """
    resultados = {}
    
    # 1. Kolmogorov-Smirnov (KS): se parametriza con la media y desviación estándar muestrales
    mu, sigma = np.mean(data), np.std(data, ddof=1)
    stat_ks, p_ks = stats.kstest(data, 'norm', args=(mu, sigma))
    resultados['KS'] = {'statistic': stat_ks, 'p_value': p_ks}
    
    # 2. Shapiro-Wilk
    stat_shapiro, p_shapiro = stats.shapiro(data)
    resultados['Shapiro'] = {'statistic': stat_shapiro, 'p_value': p_shapiro}
    
    # 3. D'Agostino-Pearson
    stat_dagostino, p_dagostino = stats.normaltest(data)
    resultados["D'Agostino"] = {'statistic': stat_dagostino, 'p_value': p_dagostino}
    
    # 4. Anderson-Darling
    ad_result = stats.anderson(data, dist='norm')
    resultados['Anderson'] = {
        'statistic': ad_result.statistic,
        'critical_values': ad_result.critical_values,
        'significance_levels': ad_result.significance_level
    }
    
    # 5. Análisis cuantitativo del Q-Q:
    # Obtenemos los cuantiles teóricos y muestrales sin ajuste (fit=False)
    quantiles_theo, quantiles_emp = stats.probplot(data, dist="norm", fit=False)
    # Realizamos una regresión lineal: cuantiles_emp = intercept + slope * cuantiles_theo
    X = sm.add_constant(quantiles_theo)
    modelo = sm.OLS(quantiles_emp, X).fit()
    intercept, slope = modelo.params
    se_intercept, se_slope = modelo.bse
    df = len(quantiles_theo) - 2
    # Prueba t para verificar si slope es 1 e intercept es 0
    t_slope = (slope - 1) / se_slope
    t_intercept = (intercept - 0) / se_intercept
    p_slope = 2 * (1 - stats.t.cdf(np.abs(t_slope), df=df))
    p_intercept = 2 * (1 - stats.t.cdf(np.abs(t_intercept), df=df))
    resultados['QQ'] = {
        'slope': slope,
        'intercept': intercept,
        'p_value_slope': p_slope,
        'p_value_intercept': p_intercept
    }
    
    return resultados

def main():
    resultados_globales = []
    total_corridas = len(combinaciones)
    
    for i, (sesgo, curtosis, varianza, outliers, tamano) in enumerate(combinaciones):
        # Generar datos simulados para la combinación actual de parámetros
        datos = generar_datos(sesgo, curtosis, varianza, outliers, tamano)
        # Aplicar las pruebas de normalidad
        resultados = test_normalidad(datos)
        # Guardar resultados junto con los parámetros usados en esta corrida
        resultados_globales.append({
            'sesgo': sesgo,
            'curtosis': curtosis,
            'varianza': varianza,
            'outliers': outliers,
            'tamano': tamano,
            'resultados': resultados
        })
        
        # Mostrar progreso cada 100 corridas
        if (i + 1) % 100 == 0:
            print(f"{i + 1} de {total_corridas} corridas completadas")
    
    # Guardar los resultados en un archivo JSON
    with open("resultados_experimento.json", "w") as f:
        json.dump(resultados_globales, f, indent=2, default=str)
    
    print("Experimento completado. Resultados guardados en 'resultados_experimento.json'.")

if __name__ == "__main__":
    main()


Total de corridas: 3125
100 de 3125 corridas completadas
200 de 3125 corridas completadas
300 de 3125 corridas completadas
400 de 3125 corridas completadas
500 de 3125 corridas completadas
600 de 3125 corridas completadas
700 de 3125 corridas completadas
800 de 3125 corridas completadas
900 de 3125 corridas completadas
1000 de 3125 corridas completadas
1100 de 3125 corridas completadas
1200 de 3125 corridas completadas
1300 de 3125 corridas completadas
1400 de 3125 corridas completadas
1500 de 3125 corridas completadas
1600 de 3125 corridas completadas
1700 de 3125 corridas completadas
1800 de 3125 corridas completadas
1900 de 3125 corridas completadas
2000 de 3125 corridas completadas
2100 de 3125 corridas completadas
2200 de 3125 corridas completadas
2300 de 3125 corridas completadas
2400 de 3125 corridas completadas
2500 de 3125 corridas completadas
2600 de 3125 corridas completadas
2700 de 3125 corridas completadas
2800 de 3125 corridas completadas
2900 de 3125 corridas completadas